### Neural Network, objective attributes data set
Using only data available prior to start the journey. This is all but the departure delay and the delay causes (weather, NSA, late plane arrival... which can only be known during or after the flight)
Will use Neural Network because it was the best perofming model in the previous excercises

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from math import sqrt

In [3]:
import sqlite3
conn=sqlite3.connect("parametres.db")
cur=conn.cursor()

In [11]:
df=pd.read_csv('delayed25.csv', index_col=False, warn_bad_lines=True, error_bad_lines=False)
# as the process take too much time, we resample the database 
df = df.sample(frac =.1)
df=df.drop(['Unnamed: 0'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48419 entries, 447482 to 400486
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ActualElapsedTime  48419 non-null  int64  
 1   AirTime            48419 non-null  int64  
 2   ArrDelay           48419 non-null  int64  
 3   DepDelay           48419 non-null  float64
 4   Distance           48419 non-null  float64
 5   TaxiIn             48419 non-null  float64
 6   TaxiOut            48419 non-null  float64
 7   Cancelled          48419 non-null  float64
 8   Diverted           48419 non-null  int64  
 9   CarrierDelay       48419 non-null  float64
 10  WeatherDelay       48419 non-null  float64
 11  NASDelay           48419 non-null  float64
 12  SecurityDelay      48419 non-null  float64
 13  LateAircraftDelay  48419 non-null  float64
 14  DepTime_sin        48419 non-null  float64
 15  DepTime_cos        48419 non-null  float64
 16  Month_sin       

##### Keep the attributes we know before flying

In [12]:
dfmin=df[['AirTime','Distance','DepTime_sin','DepTime_cos','Month_sin','Month_cos',
           'DayOfWeek_sin','DayOfWeek_cos','origin_freq','dest','carrier_me','ArrDelay']]

In [13]:
dfmin.head(3)

,AirTime,Distance,DepTime_sin,DepTime_cos,Month_sin,Month_cos,DayOfWeek_sin,DayOfWeek_cos,origin_freq,dest,carrier_me,ArrDelay
447482,199,1.733333,-0.526956,-0.849893,1.000000e+00,6.123234e-17,0.433884,-0.900969,0.027732,0.017956,36.445019,19
78489,163,1.053030,-0.710799,-0.703395,1.224647e-16,-1.000000e+00,-0.781831,0.623490,0.005420,0.024699,30.087884,-10
266391,42,-0.578788,-0.627284,-0.778791,1.000000e+00,6.123234e-17,-0.433884,-0.900969,0.049265,0.022669,46.561553,14


##### We wantto predict the possibility of a late arrival

In [14]:
target_column = ['ArrDelay'] 
predictors = list(set(list(dfmin.columns))-set(target_column))
dfmin[predictors] = dfmin[predictors]/dfmin[predictors].max()

/Users/psans/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [15]:
#Split the data into train and test sets
X = df[predictors].values
y = df[target_column].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
print(X_train.shape); print(X_test.shape)

(33893, 11)
(14526, 11)


In [16]:
# Define model previously tested

model = Sequential()
model.add(Dense(100, input_dim=11, activation='relu'))
# check to see if the regression node should be added
model.add(Dense(1))

model.summary() #Print model Summary

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               1200      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 1,301
Trainable params: 1,301
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])

model.fit(X_train, y_train, epochs=200)

In [18]:
pred_train= model.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train)))

pred= model.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred))) 

52.57064816251308
53.1841595934956


In [19]:
from sklearn.metrics import mean_absolute_error
y_pred = model.predict(X_test)

mae=mean_absolute_error(y_test, y_pred)
print('MAE: %.4f' % mae)

mse = mean_squared_error(y_test, y_pred)
print('MSE: %.3f' % mse)
print('RMSE: %.3f' % (mse*(1/2.0)))

from sklearn.metrics import r2_score
y_pred = model.predict(X_test)
r2 = r2_score(y_test,y_pred)
print('R-Squared: %.4f' % r2)

MAE: 35.1097
MSE: 2828.555
RMSE: 1414.277
R-Squared: 0.1285


In [20]:
#SQL database update
code='objective'
process='Neural_network'
datab='objective_dt'
depdelay='no'

cur.execute("INSERT OR REPLACE INTO parametres(codi,proces,DepDelay, base_dades, MAE, R2, MSE) VALUES(?,?,?,?,?,?,?)",(code,process,depdelay,datab,'%.3f' % mae,'%.3f' % r2,'%.3f' % mse))
conn.commit()

##### The compiled database with the different tests.
The last row is the result of this objective prediction, which indicates it is difficult to predict if a flight is going to be late

In [21]:
print(pd.read_sql_query("SELECT * from parametres", conn))

         codi          proces DepDelay    base_dades     MAE     R2       MSE
0         dt1   Decision_Tree      yes      standard   5.826  0.977    83.016
1         dt2   Decision_Tree      yes      standard   9.383  0.936   205.708
2         dt3   Decision_Tree       no      standard  15.117  0.840   524.455
3         dt4   Decision_Tree      yes  standard/min   9.383  0.840   205.708
4         dt5   Decision_Tree      yes   no standard   9.818  0.840   227.035
5         dt6   Decision_Tree       no   no standard  14.717  0.840   493.721
6         nn1  Neural_network      yes      standard   2.295  0.995    16.489
7         nn2  Neural_network       no      standard   2.822  0.989    35.978
8         ls1           Lasso      yes      standard   3.548  0.987    44.190
9         ls2           Lasso       no      standard   3.548  0.987    44.190
10  objective  Neural_network       no  objective_dt  35.110  0.129  2828.555
